In [208]:
import tensorflow as tf
from tensorflow import keras
import csv
import os
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [209]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(9216,)),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dense(2048, activation='relu'),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dense(128)
])

In [210]:
import random

def noisyCount(sensitivety, epsilon,u1,u2):
    beta = sensitivety/epsilon
    
    if u1 <= 0.5:
        n_value = -beta*np.log(1.-u2)
    else:
        n_value = beta*np.log(u2)
    # print(n_value)
    return n_value

def laplace_mech(data, sensitivety, epsilon,u1,u2):
    for i in range(len(data)):
            data[i] += noisyCount(sensitivety, epsilon,u1,u2)
    return data
sensitivety = 1
epsilon = 4
u:list[list]=[]
for _ in range(400):
    u.append((random.random(),random.random()))
zpk = [[random.random() for _ in range(512)] for i in range(512)]
def loadData(num, name):
    data = []
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(num): # 文件夹的数量
        # 计算数量
        files = os.listdir("./result_data/" + str(i))
        count = 0
        for f in files:
            if f.startswith("feature"):
                count += 1
        # print(count)
        for j in range(count):
            csv_reader = csv.reader(open("./result_data/" + str(i) + "/" + str(name) + str(j) + ".csv"))
            f = []
            for row in csv_reader:
                tmp = list(map(lambda x:float(x),row))
                for t in tmp:
                    f.append(t)
            #print(len(f))
            f = laplace_mech(f, sensitivety, epsilon,u[i][0],u[i][1])
            #print(f)
            #f = np.matmul(np.array(f).reshape(18,512),zpk).reshape(9216)
            print(f)
            # print(len(f))
            if count<=6:
                x_train.append(f)
                y_train.append(i)
            elif count >6 :
                if j <=6:
                    x_train.append(f)
                    y_train.append(i)
                elif j >6:
                    x_test.append(f)
                    y_test.append(i)
                
    return x_train, y_train, x_test, y_test
train_images, train_labels, test_images, test_labels=loadData(10,"feature")

[1.45610298 1.2275556  1.47085836 ... 0.91482753 0.89941221 0.92852449]
[1.50241633 1.19304891 1.44495589 ... 0.88720219 0.77833595 0.84045551]
[1.46383991 1.16082134 1.47239706 ... 0.88693067 0.88169834 0.91839786]
[1.5549577  1.28952371 1.51511803 ... 0.88620608 0.86822888 0.89438237]
[1.61627993 1.28583585 1.56582289 ... 0.83398192 0.77684288 0.88384503]
[1.41523541 1.2717862  1.41650137 ... 0.93693619 0.92351594 0.96690493]
[1.63512242 1.34572346 1.48268587 ... 0.90952368 0.92477283 1.00924221]
[1.42433377 1.18570709 1.40966902 ... 0.95871684 1.00036389 1.02187568]
[ 0.6100206   0.23635635  0.54463289 ... -0.08531978 -0.13095292
 -0.1467111 ]
[ 0.58537066  0.2614265   0.46117938 ... -0.06638983 -0.06807703
 -0.11740125]
[ 0.52523474  0.1521943   0.48487061 ... -0.11933272 -0.15426125
 -0.17879566]
[ 0.58611733  0.2705918   0.4419062  ... -0.0594427  -0.09658627
 -0.0913907 ]
[ 0.5131194   0.32681809  0.44973896 ... -0.12830707 -0.12929719
 -0.15359846]
[ 0.43958918  0.25616985  0.4

In [211]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [212]:
history = model.fit(train_images, train_labels, epochs=100)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(history.history['accuracy'])
print('\nTest accuracy:', test_acc)
# acc = history.history['accuracy']
# test_acc = history.history['val_accuracy']
# plt.plot(60,acc,'bo',label='Training acc')
# plt.plot(60,test_acc,'b',label='Validation acc')
# plt.legend()
# plt.show()

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>"}), (<class 'list'> containing values of types {"<class 'int'>"})